In [78]:
!pip install nltk gensim openai

In [79]:
import pandas as pd
import time
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords
from groq import Groq
import os
from dotenv import load_dotenv
from nltk.tokenize import sent_tokenize
import json

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oskarroeske/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# EDA

In [80]:
df_reports = pd.read_csv("preprocessed_reports.csv")
df_paragraphs = pd.read_csv("preprocessed_paragraphs.csv")

In [81]:
df_paragraphs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12573 entries, 0 to 12572
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    12573 non-null  int64 
 1   filename      12573 non-null  object
 2   document_id   12573 non-null  int64 
 3   paragraph_id  12573 non-null  int64 
 4   paragraph     12573 non-null  object
dtypes: int64(3), object(2)
memory usage: 491.3+ KB


In [82]:
df_paragraphs

,Unnamed: 0,filename,document_id,paragraph_id,paragraph
0,0,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,1,"Last night, ahead of the 2014 CES, NVDA hosted..."
1,1,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,2,GameStream works hand in hand with the PC gami...
2,2,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,3,NVDA sought to bridge the gap between PC and m...
3,3,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,4,Recognizing the shift to advanced automobile f...
4,4,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,5,NVDA has spent heavily on its mobile applicati...
...,...,...,...,...,...
12568,12568,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_Ch...,203,1,Even though available at quite at a lag with t...
12569,12569,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_Ch...,203,2,"Coming back to the October data, International..."
12570,12570,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_Ch...,203,3,International shipments Apple till October hav...
12571,12571,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_Ch...,203,4,5G mix accounted for 80% of total shipments in...


In [83]:
def clean_text_again(text):
    text = strip_tags(text)
    text = strip_multiple_whitespaces(text)  # Normalize whitespaces
    return text

In [84]:
df_paragraphs["paragraph"] = df_paragraphs["paragraph"].apply(clean_text_again)

# LLM Argument Mining

In [85]:
list_of_arguments = [
    "Growth",
    "Price/Earnings Ratio",
    "Earnings per Share",
    "Cash Flow",
    "Revenue",
    "Return On Equity",
    "Margins",
    "Cost Management",
    "Dividend Policy",
    "Investments",
    "Balance Sheet",
    "Long-term Growth",
    "Mergers and Acquisition",
    "Refranchising",
    "Sustainability",
    "Employees",
    "Research and Development",
    "Marketing",
    "Shares Repurchase",
    "Processes",
    "Leadership",
    "Innovation",
    "Product Characteristics",
    "Pricing Strategy",
    "Production",
    "Technology Trends",
    "Market Share",
    "Market Conditions",
    "Market Expansion",
    "Competition",
    "Global Presence",
    "Industry Outlook",
    "Regulations",
    "Partnerships and Collaborations",
    "Supply Chain",
    "Economic Conditions",
    "Demand",
    "Customers"
]


In [86]:
len(list_of_arguments)

38

In [87]:
from openai import OpenAI
import json

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=api_key)

def find_arguments(text):
    
    amount_of_categories = len(list_of_arguments)

    categories_string = ', '.join(f'"{cat}"' for cat in list_of_arguments)

    system_content = f"""You will analyze a paragraph from a financial analyst report to identify justifications for the companies stock recommendation. Provide a minimal JSON output summarizing the arguments with the existing categories and sentiment.
 Guidelines:
    1. Argument Identification:
    - Extract only statements that serve as justifications for the stock recommendation.
    - Ignore brand names, target prices, recommendations, or valuation as arguments.
    - If no argument is found return an empty JSON
    2. Category Assignment:
    - Read the existing categories very carefully.
    - **If {amount_of_categories} is 38 or more:**
        - **Assign argument to an existing category, even if imperfect!**
        - Do NOT create a new category.
    - **If {amount_of_categories} is less than 38:**
        - Assign identified arguments to existing categories as much as possible.
        - If the argument could fit into a broader category instead of creating a new one, prefer the broader category (e.g., 'Ad Revenue'/'Mobile Revenue' should map to 'Revenue'.
        ** If you create a new Category, it must be generic and short (maximum 2 words).**
    3. Sentiment Classification:
    - Label each argument’s sentiment as 'positive', 'negative', or 'neutral' depending on the context it is used (financial perspective).
    4. Output:
    - Return a JSON (Structure: {{'Category 1': 'negative', 'Category 2': 'positive'}}).
    5. Example:
    - {{'Revenue':'positive','Dividend Policy':'negative'}}.
   """
    user_content = f"""
        Existing Categories:[{categories_string}]
        Text: {text}
        Amount of Categories: {amount_of_categories} (MAXIMUM!: 38)
       """
    #print(user_content)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system", 
                "content":[
                    {
                        "type":"text",
                        "text":system_content
                    }
                ] 
            },
            {
                "role": "user", 
                "content":[
                    {
                        "type":"text",
                        "text":user_content
                    }
                ] 
            }
        ],
        temperature=0,
        response_format={
        "type": "json_object"
        }
    )
    #print(response)
    return response.choices[0].message.content

In [88]:
def extract_arguments_and_sentiment(commentary):
    global list_of_arguments  # Ensure we can modify the global categories list
    
    # Call the find_arguments function with the selected API key
    result_json = find_arguments(commentary)   
    
    try:
        # Parse the JSON response
        result_dict = json.loads(result_json)
        
        # Check for new categories and add them to the global list
        for category in result_dict.keys():
            if category not in list_of_arguments:
                list_of_arguments.append(category)  # Dynamically update the categories list
        
        time.sleep(1.5)
        return result_dict  # Return the result
    except json.JSONDecodeError:
        print("Failed to decode JSON response.")
        return None


In [89]:
df_paragraphs["provided_arguments"] = df_paragraphs["paragraph"].apply(extract_arguments_and_sentiment)

In [90]:
len(list_of_arguments)

274

In [91]:
df_paragraphs

,Unnamed: 0,filename,document_id,paragraph_id,paragraph,provided_arguments
0,0,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,1,"Last night, ahead of the 2014 CES, NVDA hosted...","{'Technology Trends': 'positive', 'Market Cond..."
1,1,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,2,GameStream works hand in hand with the PC gami...,"{'Technology Trends': 'positive', 'Partnership..."
2,2,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,3,NVDA sought to bridge the gap between PC and m...,"{'Technology Trends': 'positive', 'Growth': 'p..."
3,3,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,4,Recognizing the shift to advanced automobile f...,"{'Innovation': 'positive', 'Market Share': 'po..."
4,4,20140114_Needham_NVDA_NVDA-_Compelling_Technol...,512,5,NVDA has spent heavily on its mobile applicati...,"{'Growth': 'negative', 'Revenue': 'negative', ..."
...,...,...,...,...,...,...
12568,12568,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_Ch...,203,1,Even though available at quite at a lag with t...,"{'Revenue': 'negative', 'Market Share': 'posit..."
12569,12569,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_Ch...,203,2,"Coming back to the October data, International...","{'Market Share': 'positive', 'Growth': 'positi..."
12570,12570,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_Ch...,203,3,International shipments Apple till October hav...,"{'Market Share': 'positive', 'Volume Trends': ..."
12571,12571,20221230_JP_Morgan_AAPL_Apple-_Public_CAICT_Ch...,203,4,5G mix accounted for 80% of total shipments in...,"{'Growth': 'positive', 'Revenue': 'positive'}"


In [106]:
df_test_document_merge = df_paragraphs

In [107]:
# Merge arguments back to one document and get rid of paragraph_id and column
df_test_document_merge.to_csv("report_arguments.csv")

In [92]:
import pandas as pd
import ast

# Initialize a list to store the exploded rows
rows = []

# Iterate through the DataFrame and expand the dictionary into rows
for index, row in df_paragraphs.iterrows():
    provided_args = row["provided_arguments"]
    # Convert string to dictionary if needed
    if isinstance(provided_args, str):
        try:
            provided_args = ast.literal_eval(provided_args)  # Convert string to dictionary
        except (ValueError, SyntaxError):
            print(f"Invalid format for provided_args at index {index}: {provided_args}")
            continue  # Skip invalid entries

    # Ensure it's a dictionary
    if isinstance(provided_args, dict):
        #print(f"Processed provided_args: {provided_args}")
        for argument, sentiment in provided_args.items():
            rows.append({
                "ID": row["document_id"],
                "Argument": argument,
                "Sentiment": sentiment
            })

# Create a new DataFrame from the exploded rows
df_result = pd.DataFrame(rows)

# Display the resulting DataFrame
print(df_result)

        ID                         Argument Sentiment
0      512                Technology Trends  positive
1      512                Market Conditions  negative
2      512                Technology Trends  positive
3      512  Partnerships and Collaborations  positive
4      512                Technology Trends  positive
...    ...                              ...       ...
26305  203                           Growth  positive
26306  203                     Market Share  positive
26307  203                    Volume Trends   neutral
26308  203                           Growth  positive
26309  203                          Revenue  positive

[26310 rows x 3 columns]


In [93]:
df_result.to_csv("gpt-4o-mini-results_v2.csv")

In [94]:
#df_result_testing = pd.read_csv("gpt-4o-mini-results.csv")

In [95]:
#df_result_testing.drop(columns=["Unnamed: 0.1","Unnamed: 0"])

In [103]:
import pandas as pd
import json
from collections import Counter, defaultdict

# Initialize a dictionary to hold counts for each key and sentiment
sentiment_counts = defaultdict(lambda: Counter({'positive': 0, 'neutral': 0, 'negative': 0}))

# Loop through each entry in the provided_arguments column
for entry in df_paragraphs["provided_arguments"]:
    # Ensure the entry is a dictionary
    if isinstance(entry, str):
        try:
            # Convert string to dictionary if it's in JSON-like format
            arguments_dict = json.loads(entry.replace("'", '"'))  # Replace single quotes with double quotes for JSON
        except json.JSONDecodeError:
            continue  # Skip any rows that can't be parsed as dictionaries
    elif isinstance(entry, dict):
        arguments_dict = entry
    else:
        continue  # Skip if entry is not a dictionary or parseable string

    # Update counts for each key and sentiment
    for key, sentiment in arguments_dict.items():
        sentiment_counts[key][sentiment] += 1

# Convert the result to a DataFrame for easier viewing
sentiment_overview_df = pd.DataFrame(sentiment_counts).T
sentiment_overview_df.columns = ["positive", "neutral", "negative"]
sentiment_overview_df.index.name = "Argument"

# Display the overview DataFrame
sentiment_overview_df.reset_index(inplace=True)
sentiment_overview_df.sort_values(by=["positive","negative","neutral"],ascending=False)
#sentiment_overview_df["positive"] = sentiment_overview_df["positive"].apply(lambda x: x.int())


,Argument,positive,neutral,negative
3,Growth,3409,75,447
7,Revenue,3390,120,738
9,Earnings per Share,1170,112,376
8,Margins,885,54,558
1,Market Conditions,642,215,790
...,...,...,...,...
191,Portfolio Management,0,1,0
192,Net Interest Margin,0,1,0
208,Political Positioning,0,1,0
231,Development,0,1,0


In [99]:
sentiment_overview_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273 entries, 0 to 272
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Argument  273 non-null    object
 1   positive  273 non-null    int64 
 2   neutral   273 non-null    int64 
 3   negative  273 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 8.7+ KB


In [100]:
sentiment_overview_df.to_csv("result_dataframe_v2.csv")